In [ ]:
import ast
import json
import os
import pickle
from pathlib import Path

import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
import pandas as pd

In [ ]:
# モデルとトークナイザーのロード
model_name = "meta-llama/Llama-2-7b"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# トークン数を記録する変数を初期化
total_input_tokens = 0
total_output_tokens = 0

# .envファイルのロード
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path("./data/.env"))

# モデルのコスト情報のロード
with open("./data/model_costs.json", "r") as f:
    model_costs = json.load(f)

# ラベルのロード
with open("./data/labels.json", "r") as f:
    labels = json.load(f)["labels"]

# 元データのデータフレームのロード
with open("data/df_batches.pkl", "rb") as f:
    df_batches = pickle.load(f)

# 元データの辞書のロード
with open("data/original_data_dict.pkl", "rb") as f:
    original_data_dict = pickle.load(f)

In [ ]:
def classify_and_evaluate(
    texts_dict: dict[str], labels: list[str], model, tokenizer
) -> dict:
    """
    文章を指定されたラベルに分類し、満足か不満かを判断する関数

    Parameters
    ----------
    texts_dict : dict[str]
        分類する文章の辞書
    labels : list[str]
        分類するラベル
    model : LlamaForCausalLM
        使用するモデル
    tokenizer : LlamaTokenizer
        使用するトークナイザー

    Returns
    -------
    dict
        分類結果と満足度の判定結果
    """
    global total_input_tokens, total_output_tokens

    results = {}
    for text_id, text in texts_dict.items():
        prompt = (
            f"与えられた文章を次のようなラベルの1つに分類してください: {labels}。\n"
            f"次に、その文章が満足か不満のどちらであるかを判断してください。\n"
            f"出力の形式は次のような辞書型にしてください: {{text_id : {{'label': 'ラベル名', 'sentiment': '満足' または '不満'}}}}。\n"
            f"文章: {text}"
        )

        inputs = tokenizer(prompt, return_tensors="pt")
        total_input_tokens += len(inputs["input_ids"][0])

        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=100)
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        total_output_tokens += len(outputs[0])

        # 結果を辞書にパースして追加
        result = ast.literal_eval(generated_text.strip().split("\n")[-1])
        results[text_id] = result

    return results

In [ ]:
# テキストの分類と評価の実行
texts_dict = {"1": "この製品は非常に良いです。", "2": "サービスが悪かったです。"}
results = classify_and_evaluate(texts_dict, labels, model, tokenizer)
print(results)